In [4]:
import random
import pandas as pd
import numpy as np
import scipy as sp
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wmcla\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
# Import the data to a df
df = pd.read_csv('data/siop_ml_train_participant.csv')

# Import the test data to a new df
eval_df = pd.read_csv('data/siop_ml_dev_participant.csv')

# Import the weighting matrix
# weights = pd.read_csv('data/weights/mvp_matrix.csv')
weights = pd.read_csv('data/weights/dummy_matrix_no_impact.csv')

# Since we want to select a specific index later for the weights, we'll set the df index here
weighting_matrix = weights.set_index('sj_item_name')

# Confirm that the data has been imported and is formatted correctly
df.head(3)

,Respondent_ID,open_ended_1,open_ended_2,open_ended_3,open_ended_4,open_ended_5,E_Scale_score,A_Scale_score,O_Scale_score,C_Scale_score,N_Scale_score
0,10446116527,"I would change my vacation week, because I am ...",I would reach out to my boss and ask him or he...,I would not go. I am a not a social person. I ...,I would ask my manager why he/she gave me such...,I would find this experience super enjoyable. ...,2.250000,3.750000,3.166667,3.750000,2.916667
1,10440100535,I would talk to my colleague and see if they w...,I would continue to work on the project that w...,I would talk to my colleague and try to talk t...,I would feel upset about the negative feedback...,I would find this experience enjoyable. I feel...,4.666667,4.416667,4.583333,5.000000,1.333333
2,10462850071,I would feel upset because perhaps I already b...,I would start working on the project now and g...,I would feel guilty about thinking about not g...,I would feel really defensive about it. I woul...,I would find it enjoyable because I would be r...,2.250000,4.750000,4.083333,4.666667,2.166667


Past and present tense using parts of speech tagging:

In [10]:
for row in df["open_ended_1"]:
    if "VBD" or "VBN" in nltk.pos_tag(row):
        past = 1
    else:
        past = 0
        
    if "VBG" or "VBP" or "VPZ" in nltk.pos_tag(row):
        present = 1
    else:
        present = 0
    
    df["past"] = past
    df["present"] = present

df.head(3)

,Respondent_ID,open_ended_1,open_ended_2,open_ended_3,open_ended_4,open_ended_5,E_Scale_score,A_Scale_score,O_Scale_score,C_Scale_score,N_Scale_score,past,present
0,10446116527,"I would change my vacation week, because I am ...",I would reach out to my boss and ask him or he...,I would not go. I am a not a social person. I ...,I would ask my manager why he/she gave me such...,I would find this experience super enjoyable. ...,2.250000,3.750000,3.166667,3.750000,2.916667,1,1
1,10440100535,I would talk to my colleague and see if they w...,I would continue to work on the project that w...,I would talk to my colleague and try to talk t...,I would feel upset about the negative feedback...,I would find this experience enjoyable. I feel...,4.666667,4.416667,4.583333,5.000000,1.333333,1,1
2,10462850071,I would feel upset because perhaps I already b...,I would start working on the project now and g...,I would feel guilty about thinking about not g...,I would feel really defensive about it. I woul...,I would find it enjoyable because I would be r...,2.250000,4.750000,4.083333,4.666667,2.166667,1,1


Word length, which is a predictor of openness to experience

In [13]:
for row in df["open_ended_1"]:
    lengths = list(map(len,row.split()))
    sumlengths = sum(i > 5 for i in lengths)
    print(sumlengths)
    #percentoverfive = sumlengths/len(row)*100

TypeError: 'int' object is not callable

Affective tagging with Loughran-McDonald Sentiment Word Lists pulled from a pdf

In [26]:
import PyPDF2

In [35]:
# creating an object 
file = open('LM_Negative.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

# print the number of pages in pdf file
print(fileReader.numPages)

#number_of_pages = tabula.read_pdf(fileReader)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()
print(page_content)

52
Loughran
-McDonald Sentiment Word Lists 
Œ Negative
  ABANDON
 
ABANDONED
 ABANDONING
 
ABANDONMENT
 ABANDONMENTS
 ABANDONS
 ABDICATED
 
ABDICATES
 ABDICATING
 ABDICATION
 ABDICATIONS
 ABERRANT
 ABERRATION
 ABERRATIONAL
 ABERRATIONS
 ABETTING
 ABNORMAL
 ABNORMALITIES
 ABNORMALITY
 
ABNORMALLY
 ABOLISH
 ABOLISHED
 
ABOLISHES
 ABOLISHING
 ABROGATE
 ABROGATED
 
ABROGATES
 ABROGATING
 
ABROGATION
 
ABROGATIONS
 ABRUPT
 ABRUPTLY
 
ABRUPTNESS
 ABSENCE
 ABSENCES
 ABSENTEEISM
 ABUSE
 ABUSED
 
ABUSES
 ABUSING
 
ABUSIVE
 ABUSIVELY
 ABUSIVENESS
 ACCIDENT
 


In [42]:
negativewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    negativewords = negativewords + page_content
#print(negativewords)
negwords = negativewords.split()

negwords = negwords[7:]
print(negwords)

['ABANDON', 'ABANDONED', 'ABANDONING', 'ABANDONMENT', 'ABANDONMENTS', 'ABANDONS', 'ABDICATED', 'ABDICATES', 'ABDICATING', 'ABDICATION', 'ABDICATIONS', 'ABERRANT', 'ABERRATION', 'ABERRATIONAL', 'ABERRATIONS', 'ABETTING', 'ABNORMAL', 'ABNORMALITIES', 'ABNORMALITY', 'ABNORMALLY', 'ABOLISH', 'ABOLISHED', 'ABOLISHES', 'ABOLISHING', 'ABROGATE', 'ABROGATED', 'ABROGATES', 'ABROGATING', 'ABROGATION', 'ABROGATIONS', 'ABRUPT', 'ABRUPTLY', 'ABRUPTNESS', 'ABSENCE', 'ABSENCES', 'ABSENTEEISM', 'ABUSE', 'ABUSED', 'ABUSES', 'ABUSING', 'ABUSIVE', 'ABUSIVELY', 'ABUSIVENESS', 'ACCIDENT', 'ACCIDENTAL', 'ACCIDENTALLY', 'ACCIDENTS', 'ACCUSATION', 'ACCUSATIONS', 'ACCUSE', 'ACCUSED', 'ACCUSES', 'ACCUSING', 'ACQUIESCE', 'ACQUIESCED', 'ACQUIESCES', 'ACQUIESCING', 'ACQUIT', 'ACQUITS', 'ACQUITTAL', 'ACQUITTALS', 'ACQUITTE', 'D', 'ACQUITTING', 'ADULTERATE', 'ADULTERATED', 'ADULTERATING', 'ADULTERATION', 'ADULTERATIONS', 'ADVERSARIAL', 'ADVERSARIES', 'ADVERSARY', 'ADVERSE', 'ADVERSELY', 'ADVERSITIES', 'ADVERSITY', '

In [47]:
for row in df["open_ended_1"]:
    negative = 0
    words = row.split()
    for word in words:
        if word.upper() in negwords:
            negative = negative + 1
    percentnegative = negative/len(row)
    df["percentneg"] = percentnegative
print(df.head())

   Respondent_ID                                       open_ended_1  \
0    10446116527  I would change my vacation week, because I am ...   
1    10440100535  I would talk to my colleague and see if they w...   
2    10462850071  I would feel upset because perhaps I already b...   
3    10460008027  I would suggest that whoever requested the tim...   
4    10459746373  I would talk to my colleague to see if he has ...   

                                        open_ended_2  \
0  I would reach out to my boss and ask him or he...   
1  I would continue to work on the project that w...   
2  I would start working on the project now and g...   
3  I would try to finish early because it's alway...   
4  I would remind my boss that I am working on th...   

                                        open_ended_3  \
0  I would not go. I am a not a social person. I ...   
1  I would talk to my colleague and try to talk t...   
2  I would feel guilty about thinking about not g...   
3  I wouldn'